In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout
from tensorflow.keras import backend as K
from tensorflow.keras import regularizers
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from sklearn.model_selection import train_test_split
import wandb
from wandb.keras import WandbCallback
import numpy as np

In [ ]:
    #Class to build the Model
class Model:
        def __init__(self, input_shape, num_classes):
            self.input_shape = input_shape
            self.num_classes = num_classes
        def build(self):

        

            input_img = Input(shape=self.input_shape)

            # AUTOENCODER: how stacked should it be/How many layers? 
            #1st layer
            x = Conv2D(
                filters=32,
                kernel_size=3, #could be changed manually
                activation='relu',
                padding='same',
                activity_regularizer=regularizers.l1(0.0001),
                kernel_initializer='he_normal'
            )(input_img)
            x = MaxPooling2D((2, 2), padding='same')(x) #max pooling layer

            #2nd layer
            x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
            x = Conv2D(
                filters=32,
                kernel_size=3, #could be changed manually
                activation='relu',
                padding='same',
                activity_regularizer=regularizers.l1(0.0001),
                kernel_initializer='he_normal',
            )(x)
            
            x = MaxPooling2D((2, 2), padding='same')(x) #may not add, try both
            #max pooling layer to give us the result of the encoding process: latent space
            encoded = Conv2D(32, (3, 3), activation='relu', padding='same')(x)


            #start of decoding, remove comment for decoding
            #x = Conv2D(32, (3, 3), activation='relu', padding='same')(encoded)
            #x = UpSampling2D((2, 2))(x)
            #x = Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same')(x)
            #x = UpSampling2D((2, 2))(x)
            #decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)


            #PRETRAINED EFFICIENTNET, could possibly use smaller but less computationaly expensive models: try,
            # note that use of LSTM is computationally more expensive and is not sure to bring better accuracy but still worth trying 
            #also note the use of tranfer learning here, but could also go with the route of training the whole model: after HP tuning with validation data ofc
            efficientnet = EfficientNetB0(weights='imagenet', include_top=False, input_shape=self.input_shape)

            for layer in efficientnet.layers[:-20]:  #freezing everything except top 20 layers, again could also go for layer.trainable=true for all layers: do we have enough computational resources?
                layer.trainable = False

            x = efficientnet(encoded) #!!!!!!!!!!!!!!!!!!!!!!!change to encoded, change dimensions in other places accordingly
            x = GlobalAveragePooling2D()(x) #helps reduce overfitting by reducing the total number of parameters in the model
            x = Dense(32,kernel_initializer='glorot_uniform')(x) #A fully connected dense layer, possibly for feature extracion, could think of adding more layers here for feature classifications in the future
            x = Dropout(0.3)(x)#for regularization, again use of tuning: maybe too much?
            output = Dense(self.num_classes, activation='softmax')(x)#fully connected dense layer with softmax activation for producing the output probabilities of classes
                #This works as the classifier in our ML pipeline thanks to softmax


            model = Model(inputs=input_img, outputs=output)
            

            model.compile(optimizer='adam', 
                        loss='categorical_crossentropy', 
                        metrics=['accuracy'],learning_rate=0.0001)

            return model
        

In [ ]:
model = Model(input_shape=(200, 200, 1) ,num_classes=2) #this part can be replaced with correct input shape
model = model.build()  